# COGS 108 - Data Checkpoint

# Names

- Sarah Borsotto
- Dhathry Doppalapudi
- Ann Luong
- Marvin Ochoa Estrada
- Niha Malhotra

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
%pip install geopandas

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(font_scale=2, style="white")

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style
plt.rcParams['figure.figsize'] = (12, 5)

# Plots latitude and longitude points on a map
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

# Converts state plane coordinates to latitude and longitude
from pyproj import Proj, transform

%config InlineBackend.figure_format = 'retina'

# Data Cleaning

First, we will read the parking ticket dataset and find out what information we can extract from it.

In [3]:
df = pd.read_csv(r"Parking_Citations.csv")

C:\Users\annlu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,7,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
print(df.columns)
df.head()

Index(['Ticket number', 'Issue Date', 'Issue time', 'Meter Id', 'Marked Time',
       'RP State Plate', 'Plate Expiry Date', 'VIN', 'Make', 'Body Style',
       'Color', 'Location', 'Route', 'Agency', 'Violation code',
       'Violation Description', 'Fine amount', 'Latitude', 'Longitude',
       'Agency Description', 'Color Description', 'Body Style Description'],
      dtype='object')


,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,...,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Agency Description,Color Description,Body Style Description
0,1103341116,12/21/2015,1251.0,NaN,NaN,CA,200304.0,NaN,HOND,PA,...,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,NaN,NaN,NaN
1,1103700150,12/21/2015,1435.0,NaN,NaN,CA,201512.0,NaN,GMC,VN,...,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,NaN,NaN,NaN
2,1104803000,12/21/2015,2055.0,NaN,NaN,CA,201503.0,NaN,NISS,PA,...,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4,NaN,NaN,NaN
3,1104820732,12/26/2015,1515.0,NaN,NaN,CA,NaN,NaN,ACUR,PA,...,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2,NaN,NaN,NaN
4,1105461453,09/15/2015,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,...,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0,NaN,NaN,NaN


Next, we are going to get rid of all of the columns that we don't need and then drop any rows that have missing values. The information that is of interest to us is:
- ticket number
- issue date
- State Plate
- Make
- Location
- Violation code
- Violation description
- Fine amount
- Latitude
- Longitude

In [5]:
df = df[['Ticket number', 'Issue Date', 'RP State Plate', 'Make', 'Location', 'Violation code', 'Violation Description', 'Fine amount', 'Latitude', 'Longitude']]
df = df.dropna()
print(df.shape)
df.head()

(15111221, 10)


,Ticket number,Issue Date,RP State Plate,Make,Location,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,12/21/2015,CA,HOND,13147 WELBY WAY,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
1,1103700150,12/21/2015,CA,GMC,525 S MAIN ST,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
2,1104803000,12/21/2015,CA,NISS,200 WORLD WAY,8939,WHITE CURB,58.0,6439997.9,1802686.4
4,1105461453,09/15/2015,CA,CHEV,GEORGIA ST/OLYMPIC,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0
5,1106226590,09/15/2015,CA,CHEV,SAN PEDRO S/O BOYD,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0


Now, we are going to extract the year that each ticket was issued to make it easier to filter the data

In [6]:
df['year'] = pd.to_datetime(df['Issue Date'], format='%m/%d/%Y').dt.year
df.head()

,Ticket number,Issue Date,RP State Plate,Make,Location,Violation code,Violation Description,Fine amount,Latitude,Longitude,year
0,1103341116,12/21/2015,CA,HOND,13147 WELBY WAY,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015
1,1103700150,12/21/2015,CA,GMC,525 S MAIN ST,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015
2,1104803000,12/21/2015,CA,NISS,200 WORLD WAY,8939,WHITE CURB,58.0,6439997.9,1802686.4,2015
4,1105461453,09/15/2015,CA,CHEV,GEORGIA ST/OLYMPIC,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0,2015
5,1106226590,09/15/2015,CA,CHEV,SAN PEDRO S/O BOYD,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015


We're only interested in looking in data from the past 5 years, so we're only going to look at entries in which the issue year is between 2018 and 2022 (inclusive)

In [7]:
df = df[df['year'].isin(range(2018, 2023)) == True]
print(df.shape)
df['year'].value_counts()

(8448834, 11)


2018    1995302
2019    1949171
2021    1567893
2022    1513752
2020    1422716
Name: year, dtype: int64

In [8]:
df.head()

,Ticket number,Issue Date,RP State Plate,Make,Location,Violation code,Violation Description,Fine amount,Latitude,Longitude,year
19951,4336729224,10/27/2018,CA,TOYT,10667 TELFAIR AVE,22500E,BLOCKING DRIVEWAY,68.0,9.999900e+04,9.999900e+04,2018
20614,4336729235,10/27/2018,CA,OTHR,10341 WOODWARD AVE,80.73.2,EXCEED 72HRS-ST,68.0,9.999900e+04,9.999900e+04,2018
20615,4336729246,10/27/2018,CA,LEXS,10650 SHERMAN GROVE AVE,80.73.2,EXCEED 72HRS-ST,68.0,6.464946e+06,1.918022e+06,2018
20616,4336729250,10/27/2018,CA,PONT,7530 SAN FERNANDO ROAD,80.73.2,EXCEED 72HRS-ST,68.0,6.454926e+06,1.898328e+06,2018
20617,4336729261,10/27/2018,CA,PONT,7530 SAN FERNANDO ROAD,5204A-,DISPLAY OF TABS,25.0,6.454926e+06,1.898328e+06,2018


Looking at `df.head` now, you can see that there are a lot of invalid latitude and longitude coordinates (longitude and latitude are 99999). You can also see that the coordinates are represented with the California Zone 5 State Plane Coordinate System instead of the regular longitude and latitude coordinates that we are used to. We need the latitude on longitude coordinates to plot the location of the tickets on a map, so for now we are only going to look at entries with valid lat/lon coordinates and convert them to the proper values.

In [9]:
df = df[df['Longitude'] != 99999]
print(df.shape)
df.head()

(7802916, 11)


,Ticket number,Issue Date,RP State Plate,Make,Location,Violation code,Violation Description,Fine amount,Latitude,Longitude,year
20615,4336729246,10/27/2018,CA,LEXS,10650 SHERMAN GROVE AVE,80.73.2,EXCEED 72HRS-ST,68.0,6.464946e+06,1.918022e+06,2018
20616,4336729250,10/27/2018,CA,PONT,7530 SAN FERNANDO ROAD,80.73.2,EXCEED 72HRS-ST,68.0,6.454926e+06,1.898328e+06,2018
20617,4336729261,10/27/2018,CA,PONT,7530 SAN FERNANDO ROAD,5204A-,DISPLAY OF TABS,25.0,6.454926e+06,1.898328e+06,2018
20618,4336729272,10/27/2018,CA,DODG,7590 GLENOAKS BLVD,80.73.2,EXCEED 72HRS-ST,68.0,6.457772e+06,1.898373e+06,2018
20619,4336729283,10/27/2018,CA,TOYT,9601 CABRINI DR,80.73.2,EXCEED 72HRS-ST,68.0,6.458568e+06,1.899818e+06,2018


Looking at `df.shape`, you can see that there are over 7 million entries. Doing the conversion for every entry would take multiple days for our computers to run so we're only going to look at 5000 entries from each year.

In [10]:
df2018 = df[df['year'] == 2018]
df2018 = df2018.head(5000)

In [11]:
df2019 = df[df['year'] == 2019]
df2019 = df2019.head(5000)

In [12]:
df2020 = df[df['year'] == 2020]
df2020 = df2020.head(5000)

In [13]:
df2021 = df[df['year'] == 2021]
df2021 = df2021.head(5000)

In [14]:
df2022 = df[df['year'] == 2022]
df2022 = df2022.head(5000)

In [15]:
df = pd.concat([df2018, df2019, df2020, df2021, df2022])
#we should have 25,000 entries
df.shape

(25000, 11)

In [16]:
df['year'].value_counts()

2018    5000
2019    5000
2020    5000
2021    5000
2022    5000
Name: year, dtype: int64

Now that we have reduced our dataset to a size that we can work with, we can convert all of the state coordinates to latitude and longitude coordinates

In [17]:
#inProj: the current coordinate system (CA Zone 5)
inProj = Proj(init='epsg:2229', preserve_units = True)
#outProj: the coordinate system that we want (latitude/longitude)
outProj = Proj(init='epsg:4326')

for index in df.index:
    x = df.loc[index, 'Latitude']
    y = df.loc[index, 'Longitude']
    new_lon, new_lat = transform(inProj, outProj, x, y)
    df.loc[index, 'Longitude'] = new_lon
    df.loc[index, 'Latitude'] = new_lat

C:\Users\annlu\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\annlu\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\annlu\AppData\Local\Temp/ipykernel_6020/1220122332.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  new_lon, 

In [18]:
df.head()

,Ticket number,Issue Date,RP State Plate,Make,Location,Violation code,Violation Description,Fine amount,Latitude,Longitude,year
20615,4336729246,10/27/2018,CA,LEXS,10650 SHERMAN GROVE AVE,80.73.2,EXCEED 72HRS-ST,68.0,34.262383,-118.320107,2018
20616,4336729250,10/27/2018,CA,PONT,7530 SAN FERNANDO ROAD,80.73.2,EXCEED 72HRS-ST,68.0,34.208173,-118.353041,2018
20617,4336729261,10/27/2018,CA,PONT,7530 SAN FERNANDO ROAD,5204A-,DISPLAY OF TABS,25.0,34.208173,-118.353041,2018
20618,4336729272,10/27/2018,CA,DODG,7590 GLENOAKS BLVD,80.73.2,EXCEED 72HRS-ST,68.0,34.208326,-118.343629,2018
20619,4336729283,10/27/2018,CA,TOYT,9601 CABRINI DR,80.73.2,EXCEED 72HRS-ST,68.0,34.212302,-118.341011,2018


# To do in EDA
- Plot all the points on a map (will probably need separate plots for each year)
- Compare the ticket locations with the average median income at those locations
- Possibly look at the make of the car
    - See if more expensive cars are ticketed more or less
- Time would also be relevant to look at
    - Will probably see less tickets in 2020 due to lockdown